---
## **DATA EXTRACTION**
---

---
### **WE USED SOURCES FROM:**


*   **FRED API - FOR MACRO DATA**
*   **BLOOMBERG - FUNDAMENTAL DATA**


*   **REDDIT - FOR SENTIMENT EXTRACTION**
*   **RSS FEED - FMP API**

*   **YAHOO FINANCE - AUXILIARY DATA, CROSS MARKET**



**NOTE :**

--> "Please view the next file in this folder **[ ST456_SENTIMENT_DATA_EXTRACTION.ipynb ]** FILE, where we had extracted **sentiments** using reddit api keys".

--> **Our final dataset can be found in this "ST456_SENTIMENT_DATA_EXTRACTION.ipynb"**



---

In [ ]:
! pip install fredapi
! pip install yfinance
! pip install pandas-datareader

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import pandas as pd
import numpy as np
import datetime as dt
from math import pi

from fredapi import Fred
import sqlite3
import yfinance as yf

# **1. Data selection & Methodology**

### **1.1. Data selection methods**


**(1) Macroeconomic Indicators**
- Macroeconomic indicators shape the external context in which firms operate. Investors monitor them to reassess both market-wide risk premia and sectoral growth outlooks.
- Inflation and monetary policy signals, including CPI, Treasury yields and the Fed Funds Rate, anchor discount rate expectations. These metrics alter how investors value distant cash flows, influencing preferences between growth and value stocks (Fabozzi, 2015; Fama, 1990).
- Employment and spending data, such as the unemployment rate and retail sales, shape forecasts of consumer demand. Positive surprises in labour or consumption figures can reprice equities upward, especially in consumer discretionary and financial sectors.
- The inclusion of weekly-normalised deltas of these macro variables allows the model to internalise the direction and velocity of economic change—an element often missed in static factor models.

**(2) Fundamental Financial Ratios**
- Fundamental indicators provide a window into a firm’s operational and financial structure. These ratios remain critical for long-horizon investors, but also hold short-term significance when markets reprice firms around earnings announcements or guidance shifts.
- Profitability metrics like ROE and net profit margin reflect earnings quality and managerial efficiency. When profitability improves concurrently with macro tailwinds, the signal becomes more powerful (Penman, 2013; Piotroski, 2000).
- Leverage indicators, such as debt-to-equity, affect a firm’s sensitivity to credit conditions. In tightening cycles, highly leveraged firms underperform due to higher interest burdens and refinancing risks (Altman, 1968).
- By converting quarterly ratios into weekly deltas, the model captures momentum in fundamentals—an aspect that mirrors analyst upgrade cycles and investor rebalancing behaviour.

**(3) Emotion and Sentiment Layer**
- Investor emotions, extracted from X posts and RSS news feeds, represent short-term belief formation under uncertainty. These signals capture not what firms are worth, but what investors think they are worth—a key distinction in behavioural finance (Barberis et al., 1998).
- Polarity and intensity scores from BERT embeddings allow the model to weight sentiment by confidence, not just frequency. Strongly polarised views (e.g. post-earnings) can act as catalysts for sharp price moves—especially when fundamentals are ambiguous.
- Unlike traditional sentiment measures that average out noise, this layer preserves emotional extremes, which are often more predictive of short-term reversals or continuations (Tetlock, 2007).

**(4) Cross-Market Echoes:**
- Cross-asset signals—notably Bitcoin and commodity price trends—serve as indirect sentiment barometers and liquidity proxies. For instance, rising Bitcoin prices may indicate risk-on positioning across speculative assets (Zhang & Yan, 2020).
- Oil prices, while rooted in supply-demand fundamentals, influence equity sectors like energy and industrials. Their movement often anticipates inflation expectations and global growth sentiment.
- By integrating these echoes via an attention mechanism, the model learns which non-equity assets matter when, thus adapting to structural shifts (e.g., rising crypto-equity correlations during periods of speculative fervour).

### **1.2. Data description**

| **Type**    | **Category**      | **Indicator**                          | **Explanation and effect on performance**                                                                                             | **Frequency** | **Data Type** | **Time Period**     | **Collection Method**            |
|-------------|-------------------|----------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------|---------------|---------------|---------------------|-----------------------------------|
| Feature     | Liquidity         | Current Ratio                          | Measures ability to pay short-term obligations; higher values reduce perceived liquidity risk                                        | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Liquidity         | Quick Ratio                            | Excludes inventory; higher values indicate stronger short-term liquidity                                                              | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Liquidity         | Cash Ratio                             | Focuses on cash-only coverage; high values enhance stability but may imply idle capital                                               | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Solvency          | Debt-to-Equity Ratio                   | High values reflect heavy leverage and potential financial risk                                                                       | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Solvency          | Debt Ratio                             | Indicates proportion of assets financed by debt; high ratios signal financial vulnerability                                           | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Solvency          | Interest Coverage Ratio                | Measures debt servicing ability; high values reduce default risk                                                                      | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Profitability     | Return on Assets (ROA)                 | Efficiency in using assets to generate net income; high ROA supports superior firm performance                                        | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Profitability     | Return on Equity (ROE)                 | Indicates return generated on shareholders’ equity; high ROE attracts equity investors                                                | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Profitability     | Net Profit Margin                      | Measures profitability after all expenses; high margins reflect strong bottom-line control                                            | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Profitability     | Operating Margin                       | Captures operational efficiency; higher values enhance earnings stability                                                             | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Profitability     | Gross Margin                           | Measures core production profitability; high margins indicate pricing power                                                           | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Valuation         | Price-to-Earnings (P/E)                | Lower P/E may suggest undervaluation; higher P/E implies growth expectations or overpricing                                           | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Valuation         | Price-to-Book (P/B)                    | Compares market value to book equity; low P/B may signal value opportunities                                                          | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Valuation         | Price-to-Sales (P/S)                   | Relates market cap to revenue; lower values may reflect discounted market pricing                                                     | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Valuation         | Price-to-Cash Flow                     | Indicates valuation relative to operating cash; low values may attract long-term investors                                            | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Valuation         | EV/EBITDA                              | Assesses firm value relative to core earnings; low ratios often preferred for acquisition targets                                     | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Valuation         | Dividend Yield                         | Measures income return; high yield may indicate maturity or undervaluation                                                            | Quarterly     | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | GDP Growth (%)                         | Reflects economic expansion; higher growth supports earnings potential and equity sentiment                                           | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | CPI (%)                                | Measures inflation; high CPI may erode purchasing power and raise rates                                                               | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | PMI                                     | Business activity index; values above 50 signal expansion and boost market confidence                                                 | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | Unemployment Rate (%)                  | High unemployment suggests weak labour demand and consumption, reducing market optimism                                               | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | Retail Sales Growth (%)                | Higher sales reflect strong consumer demand, benefiting revenue generation                                                            | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | Net Export Growth (%)                  | Indicates trade performance; positive growth supports export-driven sectors                                                           | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | IIP (Industrial Production Index)      | Tracks real output in goods-producing sectors; growth supports cyclical equities                                                      | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | 10-year Treasury Yield (%)             | Higher yields raise discount rates, pressuring valuations, especially for growth stocks                                               | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | Saving Rate (%)                        | High saving rates reduce consumption, potentially slowing revenue growth                                                              | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | New Home Sales Growth (%)              | Higher growth signals economic confidence and real estate strength                                                                    | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Macroeconomic     | Fed Funds Rate (%)                     | Higher rates increase cost of capital and reduce equity attractiveness                                                                | Monthly       | Numeric       | 01/2009 – 12/2024    | Bloomberg Terminal                 |
| Feature     | Sentiment         | Weekly Sentiment Score (RSS + Tweets)  | Aggregated weekly sentiment score extracted via BERT from financial news headlines and social media posts                             | Weekly        | Numeric       | 01/2009 – 12/2024    | X API, FinancialModelingPrep RSS  |
| Feature     | Cross-Market      | Oil Price Weekly Change (%)            | Rising oil may reflect inflation expectations or demand rebound; relevant for energy and industrials                                  | Weekly        | Numeric       | 01/2009 – 12/2024    | Yahoo Finance                      |
| Feature     | Cross-Market      | Gold Price Weekly Change (%)           | Often inversely correlated with risk appetite; a safe-haven asset under macro uncertainty                                             | Weekly        | Numeric       | 01/2009 – 12/2024    | Yahoo Finance                      |
| Feature     | Cross-Market      | USD/VND Exchange Rate (%)              | Currency trends affect multinationals and inflation-sensitive sectors                                                                 | Weekly        | Numeric       | 01/2009 – 12/2024    | Yahoo Finance                      |
| Feature     | Cross-Market      | VIX Index Change (%)                   | Measures market volatility; spikes often precede broad risk-off behaviour                                                             | Weekly        | Numeric       | 01/2009 – 12/2024    | Yahoo Finance                      |
| Feature     | Cross-Market      | DXY (Dollar Index) Weekly Change (%)   | Reflects USD strength; relevant for global firms and commodity-linked equities                                                        | Weekly        | Numeric       | 01/2009 – 12/2024    | Yahoo Finance                      |



## **EXTRACTING MACROECONOMIC DATA**

In [ ]:
# Initialize Fred API with your key
fred_key = 'e9bc109577b3480299a5db51dcb0969e'
fred = Fred(api_key=fred_key)

# Define the series IDs for the indicators from fred website
series_ids = {
    'GDP Growth': 'A191RL1Q225SBEA',
    'CPI': 'CPALTT01USM657N',
    'PMI': 'CUUR0000SA0R',
    'Unemployment Rate':'UNRATE',
    'Retail sales':'RSAFS'
}

# Fetch data for each series and filter by date range
start_date = '1992-01-01'
end_date = '2025-01-04'
data = {}

for indicator, series_id in series_ids.items():
    try:
        data[indicator] = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
    except Exception as e:
        print(f"Error fetching data for {indicator}: {e}")

# Convert to DataFrame
df = pd.DataFrame(data)
df

In [ ]:
df.bfill(inplace=True)
df = df[:-1]
df
output_path = 'macroeconomic_indicators.csv'
df.to_csv(output_path)
print("Succesfully loaded data into macroeconomic_indicators.csv")

In [ ]:
# Define the series IDs for the indicators
series_ids = {
    'US Corporate as % of GDP': 'CP',
    'Net Export Growth': 'NETEXP',
    'Industrial Production (IIP)': 'INDPRO',
    '10-Year Government Bond Rate': 'IRLTLT01USM156N',
    'Saving Rate of People': 'PSAVERT',
    'New Home Sales': 'HSN1F',
    'FED Fund Rate': 'FEDFUNDS'
}

start_date = '1992-01-01'
end_date = '2025-01-04'
db_path = "fred_data.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS fred_data (
    date TEXT PRIMARY KEY,
    CP REAL,
    NETEXP REAL,
    INDPRO REAL,
    IRLTLT01USM156N REAL,
    PSAVERT REAL,
    HSN1F REAL,
    FEDFUNDS REAL
)
""")
all_data = {}

for indicator, series_id in series_ids.items():
    try:
        data = fred.get_series(series_id, observation_start=start_date, observation_end=end_date)
        all_data[indicator] = data
    except Exception as e:
        print(f"Error fetching data for {indicator}: {e}")

# Convert all fetched data to a DataFrame
df = pd.DataFrame(all_data)
df.index.name = 'date'
df


In [ ]:
# Save DataFrame to the SQLite database
df.reset_index(inplace=True)
df.to_sql('fred_data', conn, if_exists='replace', index=False)

print("Succesfully inserted data into SQLite database.")
conn.close()

In [ ]:
# Reconnect to the database
conn = sqlite3.connect(db_path)
query = """
SELECT
    date,
    "US Corporate as % of GDP",
    "Net Export Growth",
    "Industrial Production (IIP)",
    "10-Year Government Bond Rate",
    "Saving Rate of People",
    "New Home Sales",
    "FED Fund Rate"
FROM fred_data
"""

df_extracted = pd.read_sql_query(query, conn)
df_extracted

In [ ]:
df_extracted['date'] = pd.to_datetime(df_extracted['date']).dt.date
df_extracted

output_path = "macro_economic_indicators_sql.csv"
df_extracted.to_csv(output_path, index=False)

print("Successfully loaded this data into economic_indicators_sql.csv")
conn.close()

In [ ]:
df_extracted.bfill(inplace=True)
df_extracted

In [ ]:
df_fred = pd.read_csv('macroeconomic_indicators.csv')
df_sql = pd.read_csv('macro_economic_indicators_sql.csv')

df_combined = pd.merge(df_fred, df_sql, left_index=True, right_index=True)

df_combined.to_csv('macro_data.csv')

## **EXTRACTING BLOOMBERG DATA**

## **LOADING DATA**

In [ ]:
# Extracting Bloomberg data from a raw DataFrame
def extract_bbg_data_long(df_raw, bbg_codes, start_year=1990):

    from functools import reduce
    import pandas as pd

    bbg_row = df_raw.iloc[4]
    stock_row = df_raw.iloc[2].copy()
    stock_row.ffill(inplace=True)

    all_results = []

    for bbg_code in bbg_codes:
        matched_indices = [i for i, val in enumerate(bbg_row) if str(val).strip().upper() == bbg_code.upper()]

        if not matched_indices:
            print(f"No columns found with BBG code '{bbg_code}'")
            continue

        stock_names = [stock_row.iloc[i] for i in matched_indices]

        df = df_raw.iloc[5:, [0] + matched_indices].copy()
        df.columns = ["Date"] + stock_names
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df[df["Date"].dt.year >= start_year]
        df.dropna(how="all", subset=df.columns[1:], inplace=True)

        df["Quarter"] = "Q" + df["Date"].dt.quarter.astype(str)
        df["Year"] = df["Date"].dt.year

        df_melted = df.melt(id_vars=["Date", "Quarter", "Year"], var_name="Stock", value_name=bbg_code)
        df_melted["Stock"] = df_melted["Stock"].str.replace(" US Equity", "", regex=False)

        # Convert BBG column to float
        df_melted[bbg_code] = pd.to_numeric(df_melted[bbg_code], errors="coerce")

        all_results.append(df_melted)

    if not all_results:
        return pd.DataFrame()

    merged_df = reduce(lambda left, right: pd.merge(
        left, right, on=["Date", "Stock", "Quarter", "Year"], how="outer"), all_results)

    merged_df = merged_df.dropna(subset=bbg_codes, how="all").reset_index(drop=True)

    return merged_df


**THESE DATASETS ARE DIRECTLY DOWNLOADED FROM BLOOMBERG BY CHOOSING OUR REQUIRED PARAMETERS FOR OUR PROJECT.**

In [ ]:
# Loading the data from `data` folder
fundamental_data_path = './data/fundamental_data.csv'
weekly_trading_data_path = './data/weekly_data.csv'
macroeconomic_data_path = './data/macro_data.csv'
cross_sectional_data_path = './data/cross_sectional_weekly_trading_data.csv'
crypto_currency_path = './data/weekly_crypto_trading_data.csv'

# Read the csv files
fundamental_data = pd.read_csv(fundamental_data_path)
weekly_trading_data = pd.read_csv(weekly_trading_data_path)
macroeconomic_data = pd.read_csv(macroeconomic_data_path)
cross_sectional_data = pd.read_csv(cross_sectional_data_path)
crypto_currency_data = pd.read_csv(crypto_currency_path)


In [ ]:
# Format the datetime columns
fundamental_data['Date'] = pd.to_datetime(fundamental_data['Date'])
fundamental_data['Month'] = fundamental_data['Date'].dt.to_period('M').astype(str)
weekly_trading_data['Date'] = pd.to_datetime(weekly_trading_data['Date'])
weekly_trading_data['Month'] = weekly_trading_data['Date'].dt.to_period('M').astype(str)

# Crypto data
crypto_currency_data['Date'] = pd.to_datetime(crypto_currency_data['Date'])

# For the macroeconomic_data & cross sectional data
macroeconomic_data['Date'] = pd.to_datetime(macroeconomic_data['Date'].astype(str),format='%Y-%d-%m',errors='coerce')
macroeconomic_data['Month'] = macroeconomic_data['Date'].dt.to_period('M').astype(str)

# cross sectional data
cross_sectional_data['Date'] = pd.to_datetime(cross_sectional_data['Date'].astype(str),format='%Y-%d-%m',errors='coerce')

/var/folders/09/3nyh_sx9151fnkzg664rq1tm0000gn/T/ipykernel_6177/1801002407.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  fundamental_data['Date'] = pd.to_datetime(fundamental_data['Date'])


In [ ]:
fundamental_data = fundamental_data.sort_values(['Stock', 'Month', 'Date']).drop_duplicates(subset=['Stock', 'Month'], keep='last').reset_index(drop=True)

## **TRADING DATA CALCULATION**

In [ ]:
weekly_trading_data

,Date,WeekOfMonth,Month,Stock,PX_LAST
0,1989-01-06,1,1989-01,AAPL,0.381
1,1989-01-13,2,1989-01,AAPL,0.386
2,1989-01-20,3,1989-01,AAPL,0.364
3,1989-01-27,4,1989-01,AAPL,0.336
4,1989-02-03,1,1989-02,AAPL,0.350
...,...,...,...,...,...
789899,2023-04-28,4,2023-04,SPX Index,5667.560
789900,2023-05-05,1,2023-05,SPX Index,5580.940
789901,2023-05-12,2,2023-05,SPX Index,5074.080
789902,2023-05-19,3,2023-05,SPX Index,5363.360


In [ ]:
weekly_trading_data = weekly_trading_data.sort_values(['Stock', 'Date'])

weekly_trading_data['LogRet'] = (
    weekly_trading_data.groupby('Stock')['PX_LAST']
    .transform(lambda x: np.log(x) - np.log(x.shift(1)))
)

In [ ]:
def classify_signal(ret, threshold=0.02):
    if ret > threshold:
        return 'BUY'
    elif ret < -threshold:
        return 'SELL'
    else:
        return 'NEUTRAL'

weekly_trading_data['Signal'] = weekly_trading_data['LogRet'].apply(classify_signal)

In [ ]:
weekly_trading_data.head()

,Date,WeekOfMonth,Month,Stock,PX_LAST,LogRet,Signal
390195,1999-10-15,3,1999-10,A,27.2035,NaN,NEUTRAL
390196,1999-10-22,4,1999-10,A,27.7509,0.019923,NEUTRAL
390197,1999-10-29,5,1999-10,A,29.9828,0.077356,BUY
390198,1999-11-05,1,1999-11,A,30.1512,0.005601,NEUTRAL
390199,1999-11-12,2,1999-11,A,30.9513,0.026190,BUY


## **CRYPTO + CROSS SECTIONAL DATA**

In [ ]:
cross_sectional_data.head()

,Date,Month,WeekOfMonth,CO1 Comdty,DXY Curncy,VIX Index,XAUUSD Curncy
0,1989-06-01,1989-01,1,16.25,94.470,NaN,406.35
1,1989-03-02,1989-02,1,16.17,96.740,NaN,390.00
2,1989-10-02,1989-02,2,15.82,96.430,NaN,389.60
3,1989-03-03,1989-03,1,16.95,95.780,NaN,386.40
4,1989-10-03,1989-03,2,17.11,96.570,NaN,393.50
...,...,...,...,...,...,...,...
742,2025-10-01,2025-01,2,79.76,109.650,19.54,2689.76
743,2025-07-02,2025-02,1,74.66,108.040,16.54,2861.07
744,2025-07-03,2025-03,1,70.36,103.838,23.37,2909.10
745,2025-04-04,2025-04,1,65.58,103.023,45.31,3038.24


In [ ]:
crypto_currency_data.head()

,Date,Month,WeekOfMonth,BTC-USD,ETH-USD
0,2014-09-15,2014-09,3,398.821014,0.000000
1,2014-09-22,2014-09,4,377.181000,0.000000
2,2014-09-29,2014-09,5,320.510010,0.000000
3,2014-10-06,2014-10,1,378.549011,0.000000
4,2014-10-13,2014-10,2,389.545990,0.000000
...,...,...,...,...,...
549,2025-03-24,2025-03,4,82334.523438,1806.218628
550,2025-03-31,2025-03,5,78214.484375,1576.728027
551,2025-04-07,2025-04,1,83684.976562,1596.685791
552,2025-04-14,2025-04,2,85174.304688,1587.514282


## **FUNDAMENTAL & MACROECONOMIC DATA**

In [ ]:
fundamental_data.head()

,Date,Quarter,Year,Stock,CUR_RATIO,QUICK_RATIO,CASH_RATIO,TOT_DEBT_TO_TOT_EQY,TOT_DEBT_TO_TOT_ASSET,INTEREST_COVERAGE_RATIO,...,PX_TO_CASH_FLOW,TRAIL_12M_NET_INC_GROWTH,TRAILING_12M_SALES_GROWTH,ASSET_GROWTH,TOTAL_EQUITY_1_YEAR_GROWTH,TRAIL_12M_NET_INC,TRAIL_12M_NET_SALES,BS_TOT_ASSET,TOTAL_EQUITY,Month
0,1996-06-28,Q2,1996,A,2.2195,0.8841,0.0000,0.0000,0.0000,2.1389,...,18.633,47.8516,16.6247,54.9901,52.236,512.0,8331.0,5050.0,3220.0,1996-06
1,1996-09-30,Q3,1996,A,2.1047,0.9726,0.0000,0.0000,0.0000,2.1389,...,18.633,47.8516,16.6247,54.9901,52.236,512.0,8331.0,5444.0,3382.0,1996-09
2,1996-12-31,Q4,1996,A,2.4103,1.3812,0.6821,9.8752,6.2333,2.1389,...,18.633,47.8516,16.6247,54.9901,52.236,569.0,8396.0,7107.0,4486.0,1996-12
3,1997-03-31,Q1,1997,A,2.3134,1.2827,0.4474,2.1112,1.3386,2.1389,...,18.633,47.8516,16.6247,54.9901,52.236,578.0,8528.0,7321.0,4642.0,1997-03
4,1997-06-30,Q2,1997,A,2.1291,1.1434,0.2801,2.6316,1.6481,2.1389,...,18.633,47.8516,16.6247,54.9901,52.236,598.0,8792.0,7827.0,4902.0,1997-06


## **AGGREGATION OF DATA**

In [ ]:
# Aggregate the data
weekly_panel = weekly_trading_data[['Stock', 'Month','WeekOfMonth']].drop_duplicates().sort_values(['Stock', 'Month','WeekOfMonth']).reset_index(drop=True)

In [ ]:
raw_data_weekly = pd.merge(weekly_panel, weekly_trading_data.drop(columns=['PX_LAST','LogRet']),on=['Stock','Month','WeekOfMonth'],how='left')

In [ ]:
# sort the fundamental data
fundamental_data = fundamental_data.sort_values(['Stock', 'Month'])

# Agregate data
raw_data_fundamental = pd.merge(raw_data_weekly, fundamental_data, on=['Stock','Month'], how='left')

raw_data_fundamental = (
    raw_data_fundamental.sort_values(['Stock', 'Month'])
    .groupby('Stock', group_keys=False)
    .apply(lambda df: df.assign(**df.drop(columns='Stock').ffill()))
    .reset_index(drop=True)
    .infer_objects(copy=False)
)

/var/folders/09/3nyh_sx9151fnkzg664rq1tm0000gn/T/ipykernel_6177/3014803646.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .apply(lambda df: df.assign(**df.drop(columns='Stock').ffill()))
/var/folders/09/3nyh_sx9151fnkzg664rq1tm0000gn/T/ipykernel_6177/3014803646.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .apply(lambda df: df.assign(**df.drop(columns='Stock').ffill()))
/var/folders/09/3nyh_sx9151fnkzg664rq1tm0000gn/T/ipykernel_6177/3014803646.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and 

In [ ]:
final_data = pd.merge(raw_data_fundamental, macroeconomic_data, on=['Month'],how='left')

In [ ]:
final_data = final_data.drop(columns=['Date_x','Date_y'])

In [ ]:
final_data.columns

Index(['Stock', 'Month', 'WeekOfMonth', 'Signal', 'Quarter', 'Year',
       'CUR_RATIO', 'QUICK_RATIO', 'CASH_RATIO', 'TOT_DEBT_TO_TOT_EQY',
       'TOT_DEBT_TO_TOT_ASSET', 'INTEREST_COVERAGE_RATIO',
       'TRAIL_12M_GROSS_MARGIN', 'TRAIL_12M_OPER_MARGIN',
       'TRAIL_12M_PROF_MARGIN', 'RETURN_ON_ASSET', 'RETURN_COM_EQY',
       'ASSET_TURNOVER', 'ACCT_RCV_TURN', 'ACCOUNTS_PAYABLE_TURNOVER_DAYS',
       'ACCT_RCV_DAYS', 'PE_RATIO', 'PX_TO_BOOK_RATIO', 'PX_TO_SALES_RATIO',
       'DIVIDEND_12_MONTH_YIELD', 'CURRENT_EV_TO_T12M_EBITDA',
       'PX_TO_CASH_FLOW', 'TRAIL_12M_NET_INC_GROWTH',
       'TRAILING_12M_SALES_GROWTH', 'ASSET_GROWTH',
       'TOTAL_EQUITY_1_YEAR_GROWTH', 'TRAIL_12M_NET_INC',
       'TRAIL_12M_NET_SALES', 'BS_TOT_ASSET', 'TOTAL_EQUITY', 'Date',
       'GDP Growth', 'CPI', 'Unemployment Rate', 'Retail sales',
       'Net Export Growth', '10-Year Government Bond Rate',
       'Saving Rate of People', 'New Home Sales', 'FED Fund Rate', 'PMI - 50',
       'IIP Growth 

In [ ]:
# Merge BTC & ETH from crypto into final_data by Month + WeekOfMonth
final_data_x = pd.merge(
    final_data,
    crypto_currency_data[['Month', 'WeekOfMonth', 'BTC-USD', 'ETH-USD']],
    on=['Month', 'WeekOfMonth'],
    how='left'
)

In [ ]:
# Merge BTC & ETH from crypto into final_data by Month + WeekOfMonth
final_data_y = pd.merge(
    final_data_x,
    cross_sectional_data[['Month', 'WeekOfMonth','CO1 Comdty', 'DXY Curncy', 'VIX Index', 'XAUUSD Curncy']],
    on=['Month', 'WeekOfMonth'],
    how='left'
)

In [ ]:
final_data_y.drop(columns=['CO1 Comdty_x','VIX Index_x','XAUUSD Curncy_x','DXY Curncy_x','BTC-USD_x','ETH-USD_x'])
final_data_y

,Stock,Month,WeekOfMonth,Signal,Quarter,Year,CUR_RATIO,QUICK_RATIO,CASH_RATIO,TOT_DEBT_TO_TOT_EQY,...,New Home Sales,FED Fund Rate,PMI - 50,IIP Growth (%),BTC-USD_y,ETH-USD_y,CO1 Comdty_y,DXY Curncy_y,VIX Index_y,XAUUSD Curncy_y
0,A,1999-10,3,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,...,0.054195,5.20,1.0,1.279586,NaN,NaN,NaN,NaN,NaN,NaN
1,A,1999-10,4,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,...,0.054195,5.20,1.0,1.279586,NaN,NaN,NaN,NaN,NaN,NaN
2,A,1999-10,5,BUY,NaN,NaN,NaN,NaN,NaN,NaN,...,0.054195,5.20,1.0,1.279586,NaN,NaN,NaN,NaN,NaN,NaN
3,A,1999-11,1,NEUTRAL,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.010375,5.42,1.0,0.533838,NaN,NaN,22.83,100.280,21.66,290.50
4,A,1999-11,2,BUY,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.010375,5.42,1.0,0.533838,NaN,NaN,24.59,100.750,21.65,291.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789899,ZTS,2024-02,3,NEUTRAL,Q3,2012.0,1.7544,0.9681,0.5824,113.0818,...,-0.032137,5.33,-1.0,1.225525,51733.238281,3112.697266,NaN,NaN,NaN,NaN
789900,ZTS,2024-02,4,NEUTRAL,Q3,2012.0,1.7544,0.9681,0.5824,113.0818,...,-0.032137,5.33,-1.0,1.225525,63167.371094,3490.993652,NaN,NaN,NaN,NaN
789901,ZTS,2024-03,1,SELL,Q3,2012.0,1.7544,0.9681,0.5824,113.0818,...,0.060350,5.33,-1.0,-0.202576,69019.789062,3881.193115,83.55,103.861,13.11,2082.92
789902,ZTS,2024-03,2,NEUTRAL,Q3,2012.0,1.7544,0.9681,0.5824,113.0818,...,0.060350,5.33,-1.0,-0.202576,68390.625000,3642.413086,82.08,102.712,14.74,2178.95


In [ ]:
# Identify all columns ending in '_y'
y_cols = [col for col in final_data_y.columns if col.endswith('_y')]

# Fill NaNs in those columns with 0
final_data_y[y_cols] = final_data_y[y_cols].fillna(0)

In [ ]:
final_data_y.to_csv('./data/cross_sectional.csv',index=False)

In [ ]:
final_df = final_data.drop(columns=['Date_x','Date_y'])
final_df = final_df.dropna(how='any')

In [ ]:
len(final_df)

511455